In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle

In [2]:
def feature_select(input_df):
    output_df = input_df[['MedianIncome', 'MedianAge', 'Family', 'Nonfamily',
       'Householdsize0', 'Householdsize2', 'Pop Age 15+: Married',
       'Pop Age 15+: Nevermarried', 'Pop Age 15+: Divorced',
       'Pop Age 25+: Bachelor\'s Degree', 'Pop Age 25 Grad Professional',
       'Adult', 'Blackpop', 'Whitepop', 'Hispanicpop', 'Mobility', 'AREA',
       'Population_Density', 'Female_density', 'GDP', 'Is_Florida',
       'Is_Tennessee', 'Is_Alabama', 'TJ.Maxx', 'Marshalls', 'HomeGoods',
       'Lowe\'s_3m', 'The Home Depot_3m', 'Walmart_3m', 'Burlington_3m',
       'Kohl\'s_3m', 'Ross Dress for Less_3m', 'Lowe\'s_visits_3m',
       'The Home Depot_visits_3m', 'Walmart_visits_3m', 'Burlington_visits_3m',
       'Kohl\'s_visits_3m', 'Ross Dress for Less_visits_3m', 'Lowe\'s_1m',
       'The Home Depot_1m', 'Burlington_1m', 'Kohl\'s_1m',
       'Ross Dress for Less_1m', 'Lowe\'s_visits_1m',
       'The Home Depot_visits_1m', 'Burlington_visits_1m', 'Kohl\'s_visits_1m',
       'Ross Dress for Less_visits_1m', 'Aldi\'s_3m', 'Publix_3m',
       'The Fresh Market_3m', 'Whole Foods Market_3m', 'Winn-Dixie_3m',
       'Aldi\'s_visits_3m', 'Publix_visits_3m', 'The Fresh Market_visits_3m',
       'Whole Foods Market_visits_3m', 'Winn-Dixie_visits_3m', 'num_star',
       'tjmaxx_3m', 'Marshalls_3m', 'Homegoods_3m', 'tjmaxx_visits_3m',
       'Marshalls_visits_3m', 'Homegoods_visits_3m']]
    return output_df

########################################################################################
def winsorize(input_df):
    import pickle
    # load the mean and standard deviation of the original data distribution
    file = open('mean.pkl', 'rb')
    df_mean = pickle.load(file)
    df_mean = df_mean.drop(labels=['Population'])
    file.close()
    
    file = open('std.pkl', 'rb')
    df_std = pickle.load(file)
    df_std = df_std.drop(labels=['Population'])
    file.close()
    
    # winsorize using 3*std
    df_1 = pd.DataFrame()
    for i in input_df.columns:
        temp = input_df[i].copy()
        lo = temp - 3*df_std[i]
        up = temp + 3*df_std[i]
        temp[temp < lo] = lo
        temp[temp > up] = up

        df_1 = pd.concat([df_1, temp], axis=1, ignore_index=True, sort=False)
    df_1.columns = input_df.columns
    return df_1

########################################################################################
def min_max(df):
    # load the min and max value of the original data distribution
    import pickle
    file = open('max.pkl', 'rb')
    df_max = pickle.load(file)
    df_max = df_max.drop(labels=['Population'])
    file.close()
    
    file = open('min.pkl', 'rb')
    df_min = pickle.load(file)
    df_min = df_min.drop(labels=['Population'])
    file.close()
    
    output_df = (df - df_min) / (df_max - df_min)
    return output_df


In [3]:
def model_output(df):
    preprocessed_df = min_max(winsorize(feature_select(df)))
    # load the model
    import pickle
    file = open('rf.pkl', 'rb')
    rf = pickle.load(file)
    file.close()
    
    return rf.predict(preprocessed_df)

In [4]:
df = pd.read_csv('dataset.csv', index_col='Unnamed: 0')

In [5]:
out = model_output(df)